In [1]:
!pip install beautifulsoup4

In [2]:
!pip install requests

In [3]:
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
from requests import get
import re

In [4]:
HEADERS={...}
drom_url = 'https://auto.drom.ru/region77/premium/'

#Отправим GET()-запрос на сайт и сохраним полученное в переменную 'page':

page = get(drom_url)

#Проверим подключение:

print(page.status_code)

#Код вернул нам статус код '200', мы успешно подключены



200


In [5]:
print(page.content)

b'<!DOCTYPE html>\n<html\n    xmlns="http://www.w3.org/1999/xhtml"\n    lang="ru"\n    xml:lang="ru"\n    class="drom-notouch no-js light no-main0"\n>\n<head>\n                    <title>\xd1\xef\xe5\xf6\xf0\xe0\xe7\xec\xe5\xf9\xe5\xed\xe8\xe5 \xe2 \xcc\xee\xf1\xea\xe2\xe5</title>\n        \n                <meta http-equiv="X-UA-Compatible" content="IE=Edge"/>\n\n        <meta name=\'copyright\' content=\'drom.ru\' />\n<meta name=\'theme-color\' content=\'#000000\' />\n<meta http-equiv=\'Content-Type\' content=\'text/html; charset=windows-1251\' />\n<meta charset=\'windows-1251\' />\n<meta name=\'candy.config\' content=\'{&quot;geor&quot;:77,&quot;geoc&quot;:0,&quot;id&quot;:27,&quot;b&quot;:1,&quot;bc&quot;:1,&quot;charset&quot;:&quot;utf-8&quot;,&quot;cf&quot;:{&quot;category_id&quot;:1}}\' />\n<meta name=\'description\' content=\'\xd7\xe0\xf1\xf2\xed\xfb\xe5 \xee\xe1\xfa\xff\xe2\xeb\xe5\xed\xe8\xff \xee \xef\xf0\xee\xe4\xe0\xe6\xe5 \xe0\xe2\xf2\xee\xec\xee\xe1\xe8\xeb\xe5\xe9 \xed\

In [6]:
#Воспользуемся BeautifulSoup4:
soup = BeautifulSoup(page.content, "html.parser")

In [7]:
#Парсим по тэгу div чтобы получить данные о всех автомобилях на вкладке Спецразмещение в Москве
all=soup.find_all("div",  class_="css-12lf65q e3qgya0")

for item in all:
  name_car=item.find_all("span", class_="css-1t71ine e16zdi0i0") #Парсим Марки авто и год выпуска по тэгу span
  name_cars=[]
  for p in name_car:
      name_cars.append(p)
  df1 = pd.DataFrame(name_car, columns = ['Марка автомобиля и год выпуска'])
  print(df1)



       Марка автомобиля и год выпуска
0              Volkswagen Tiguan 2020
1          Mercedes-Benz E-Class 2018
2                 Opel Astra GTC 2008
3                  BYD Song Plus 2023
4               Volkswagen Tharu 2022
5              Mercedes-Benz GLE 2019
6         Mercedes-Benz GL-Class 2013
7           Mitsubishi Outlander 2022
8               Hyundai Palisade 2019
9                   Cadillac ATS 2016
10                Chery Arrizo 8 2023
11  Land Rover Range Rover Sport 2008
12                Chery Arrizo 8 2023
13                        BMW X6 2020
14               Lincoln Corsair 2021
15               Hyundai Solaris 2021
16                 Geely Monjaro 2023
17                  Nissan Rogue 2018
18                 Geely Monjaro 2023
19                 Porsche Macan 2020
20                       Zeekr X 2023
21                        BMW X7 2023
22                   Лада Приора 2011
23                 Changan UNI-V 2023
24                Chery Arrizo 8 2023
25          

In [8]:
#Парсим Стоимость авто по тэгу div
cost_cars_all = []
for x in soup.find_all('div', class_='css-aimy81 egua2dj0'):

# Очищаем данные
  x1 = re.sub("[^0-9]", "", str(x))
  x2=x1.replace("8120", "")
  x2_s= int(x2)

  cost_cars_all.append(x2_s)
df2 = pd.DataFrame(cost_cars_all, columns = ['Стоимость автомобиля'])
df2

,Стоимость автомобиля
0,3830000
1,2300000
2,400000
3,3700000
4,3490000
5,7500000
6,2990000
7,3150000
8,4800000
9,1780000


In [9]:
# объединяем DataFrames
df_full = pd.concat([df1, df2], join = 'outer', axis = 1)
print (df_full)

       Марка автомобиля и год выпуска  Стоимость автомобиля
0              Volkswagen Tiguan 2020               3830000
1          Mercedes-Benz E-Class 2018               2300000
2                 Opel Astra GTC 2008                400000
3                  BYD Song Plus 2023               3700000
4               Volkswagen Tharu 2022               3490000
5              Mercedes-Benz GLE 2019               7500000
6         Mercedes-Benz GL-Class 2013               2990000
7           Mitsubishi Outlander 2022               3150000
8               Hyundai Palisade 2019               4800000
9                   Cadillac ATS 2016               1780000
10                Chery Arrizo 8 2023               3349900
11  Land Rover Range Rover Sport 2008               1450000
12                Chery Arrizo 8 2023               3349900
13                        BMW X6 2020               9700000
14               Lincoln Corsair 2021               3450000
15               Hyundai Solaris 2021   